In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/wonka_choc_factory.csv")

In [3]:
# use the existing normalized column names (Order Date, Ship Date)
# (do not re-import pandas if it was already imported in another cell)
df["Order Date"] = pd.to_datetime(df.get("Order Date"), errors="coerce")
df["Ship Date"] = pd.to_datetime(df.get("Ship Date"), errors="coerce")

# Fix: subtract 5 years from all ship dates
df["Ship Date"] = df["Ship Date"] - pd.DateOffset(years=5)

# Optional: check delays now
df["Fulfillment Days"] = (df["Ship Date"] - df["Order Date"]).dt.days
print(df["Fulfillment Days"].describe())


count    10194.000000
mean       177.714244
std          1.803791
min        173.000000
25%        177.000000
50%        178.000000
75%        179.000000
max        185.000000
Name: Fulfillment Days, dtype: float64


In [4]:
from pandas.tseries.offsets import DateOffset

# --- 1. Identify Column Names ---
if "Order_Date" in df.columns:
    od_col = "Order_Date"
elif "Order Date" in df.columns:
    od_col = "Order Date"
else:
    raise KeyError("Order date column not found in dataframe")

# Set the Ship Date column name (assuming 'Ship_Date' is the normalized one)
if "Ship_Date" in df.columns:
    sd_col = "Ship_Date"
elif "Ship Date" in df.columns:
    sd_col = "Ship Date"
else:
    sd_col = "Ship_Date"

# --- 2. Ensure Both Columns are Datetime Objects ---
df[od_col] = pd.to_datetime(df[od_col], errors="coerce")
df[sd_col] = pd.to_datetime(df[sd_col], errors="coerce")

# --- 3. APPLY THE 5-YEAR CORRECTION (The crucial step) ---
mask_erroneous_year = df[sd_col].dt.year >= 2027  # Check for 2027 or later

df.loc[mask_erroneous_year, sd_col] = (
    df.loc[mask_erroneous_year, sd_col] - DateOffset(years=5)
)

# --- 4. Compute Delay and Show Summary ---
df["Fulfillment Days"] = (df[sd_col] - df[od_col]).dt.days
print(df["Fulfillment Days"].describe())

count    10194.000000
mean       177.714244
std          1.803791
min        173.000000
25%        177.000000
50%        178.000000
75%        179.000000
max        185.000000
Name: Fulfillment Days, dtype: float64


In [23]:
df["Fulfillment Days"].sort_values(ascending=False).head(10)

5051    185
5052    185
167     182
166     182
1088    181
1087    181
1086    181
6726    181
397     181
1372    181
Name: Fulfillment Days, dtype: int64

In [24]:
(df["Fulfillment Days"] > 365).sum()

np.int64(0)

In [ ]:
df.to_csv("wonka_choc_factory_clean.csv",index=False)

In [27]:
import os
os.getcwd()

'/Users/lee/Documents/GitHub/SQL-Project-Willy-Wonka-Bain-and-Co/willy-wonka-project/notebooks'

In [5]:
ship_mode_counts = (
    df["Ship Mode"]
    .value_counts()
    .reset_index()
)

ship_mode_counts.columns = ["Ship Mode", "total_orders"]
ship_mode_counts

,Ship Mode,total_orders
0,Standard Class,6120
1,Second Class,1979
2,First Class,1548
3,Same Day,547


In [6]:
df.head(10)

,Unnamed: 0,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State/Province,Postal Code,Division,Region,Product Name,Sales,Units,Gross Profit,Cost,Factory,Latitude,Longitude,Fulfillment Days
0,0,2021-03-31,2021-09-26,Standard Class,128055,United States,San Francisco,California,94122,Chocolate,Pacific,Wonka Bar - Triple Dazzle Caramel,7.50,2,4.90,2.60,Wicked Choccy's,32.076176,-81.088371,179
1,1,2021-03-31,2021-09-26,Standard Class,128055,United States,San Francisco,California,94122,Chocolate,Pacific,Wonka Bar -Scrumdiddlyumptious,7.20,2,5.00,2.20,Lot's O' Nuts,32.881893,-111.768036,179
2,2,2021-09-15,2022-03-13,Standard Class,138100,United States,New York City,New York,10011,Chocolate,Atlantic,Wonka Bar - Fudge Mallows,7.20,2,4.80,2.40,Lot's O' Nuts,32.881893,-111.768036,179
3,3,2021-09-15,2022-03-13,Standard Class,138100,United States,New York City,New York,10011,Chocolate,Atlantic,Wonka Bar - Milk Chocolate,9.75,3,6.33,3.42,Wicked Choccy's,32.076176,-81.088371,179
4,4,2022-10-04,2023-03-29,First Class,121391,United States,San Francisco,California,94109,Chocolate,Pacific,Wonka Bar - Milk Chocolate,6.50,2,4.22,2.28,Wicked Choccy's,32.076176,-81.088371,176
5,5,2023-03-03,2023-08-28,Standard Class,103982,United States,Round Rock,Texas,78664,Chocolate,Interior,Wonka Bar -Scrumdiddlyumptious,3.60,1,2.50,1.10,Lot's O' Nuts,32.881893,-111.768036,178
6,6,2023-03-03,2023-08-28,Standard Class,103982,United States,Round Rock,Texas,78664,Chocolate,Interior,Wonka Bar - Nutty Crunch Surprise,10.47,3,7.47,3.00,Lot's O' Nuts,32.881893,-111.768036,178
7,7,2023-03-03,2023-08-28,Standard Class,103982,United States,Round Rock,Texas,78664,Chocolate,Interior,Wonka Bar - Fudge Mallows,25.20,7,16.80,8.40,Lot's O' Nuts,32.881893,-111.768036,178
8,8,2023-03-03,2023-08-28,Standard Class,103982,United States,Round Rock,Texas,78664,Chocolate,Interior,Wonka Bar - Milk Chocolate,9.75,3,6.33,3.42,Wicked Choccy's,32.076176,-81.088371,178
9,9,2024-06-29,2024-12-25,Standard Class,147039,United States,Minneapolis,Minnesota,55407,Chocolate,Interior,Wonka Bar - Nutty Crunch Surprise,10.47,3,7.47,3.00,Lot's O' Nuts,32.881893,-111.768036,179


In [7]:
df.shape

(10194, 20)

In [8]:
df.tail(10)

,Unnamed: 0,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State/Province,Postal Code,Division,Region,Product Name,Sales,Units,Gross Profit,Cost,Factory,Latitude,Longitude,Fulfillment Days
10184,10184,2024-11-06,2025-05-04,Standard Class,100013,United States,Los Angeles,California,90045,Chocolate,Pacific,Wonka Bar -Scrumdiddlyumptious,7.20,2,5.00,2.20,Lot's O' Nuts,32.881893,-111.768036,179
10185,10185,2021-08-27,2022-02-22,Second Class,143336,United States,San Francisco,California,94109,Chocolate,Pacific,Wonka Bar - Triple Dazzle Caramel,7.50,2,4.90,2.60,Wicked Choccy's,32.076176,-81.088371,179
10186,10186,2021-08-27,2022-02-22,Second Class,143336,United States,San Francisco,California,94109,Chocolate,Pacific,Wonka Bar - Triple Dazzle Caramel,15.00,4,9.80,5.20,Wicked Choccy's,32.076176,-81.088371,179
10187,10187,2021-08-27,2022-02-22,Second Class,143336,United States,San Francisco,California,94109,Chocolate,Pacific,Wonka Bar -Scrumdiddlyumptious,10.80,3,7.50,3.30,Lot's O' Nuts,32.881893,-111.768036,179
10188,10188,2023-04-03,2023-09-29,Standard Class,167682,United States,Richmond,Indiana,47374,Chocolate,Interior,Wonka Bar - Milk Chocolate,13.00,4,8.44,4.56,Wicked Choccy's,32.076176,-81.088371,179
10189,10189,2023-04-03,2023-09-29,Standard Class,167682,United States,Richmond,Indiana,47374,Chocolate,Interior,Wonka Bar -Scrumdiddlyumptious,14.40,4,10.00,4.40,Lot's O' Nuts,32.881893,-111.768036,179
10190,10190,2023-05-05,2023-10-29,Standard Class,147991,United States,Chattanooga,Tennessee,37421,Chocolate,Gulf,Wonka Bar - Nutty Crunch Surprise,17.45,5,12.45,5.00,Lot's O' Nuts,32.881893,-111.768036,177
10191,10191,2023-07-08,2023-12-28,Same Day,152471,United States,Jacksonville,Florida,32216,Chocolate,Gulf,Wonka Bar - Triple Dazzle Caramel,7.50,2,4.90,2.60,Wicked Choccy's,32.076176,-81.088371,173
10192,10192,2023-07-08,2023-12-28,Same Day,152471,United States,Jacksonville,Florida,32216,Chocolate,Gulf,Wonka Bar - Triple Dazzle Caramel,18.75,5,12.25,6.50,Wicked Choccy's,32.076176,-81.088371,173
10193,10193,2024-06-11,2024-12-05,First Class,141481,United States,Los Angeles,California,90036,Chocolate,Pacific,Wonka Bar - Fudge Mallows,10.80,3,7.20,3.60,Lot's O' Nuts,32.881893,-111.768036,177


In [9]:
df.columns

Index(['Unnamed: 0', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID',
       'Country/Region', 'City', 'State/Province', 'Postal Code', 'Division',
       'Region', 'Product Name', 'Sales', 'Units', 'Gross Profit', 'Cost',
       'Factory', 'Latitude', 'Longitude', 'Fulfillment Days'],
      dtype='object')

In [10]:
df["Ship Mode"].value_counts()

Ship Mode
Standard Class    6120
Second Class      1979
First Class       1548
Same Day           547
Name: count, dtype: int64

In [11]:
df["Country/Region"].value_counts()

Country/Region
United States    9994
Canada            200
Name: count, dtype: int64

In [12]:
df["City"].value_counts()

City
New York City    915
Los Angeles      747
Philadelphia     537
San Francisco    510
Seattle          428
                ... 
Springdale         1
Layton             1
Montebello         1
Deer Park          1
Portage            1
Name: count, Length: 542, dtype: int64

In [13]:
df["City"].nunique()

542

In [14]:
df["State/Province"].value_counts()

State/Province
California                   2001
New York                     1128
Texas                         985
Pennsylvania                  587
Washington                    506
Illinois                      492
Ohio                          469
Florida                       383
Michigan                      255
North Carolina                249
Arizona                       224
Virginia                      224
Georgia                       184
Tennessee                     183
Colorado                      182
Indiana                       149
Kentucky                      139
Massachusetts                 135
New Jersey                    130
Oregon                        124
Wisconsin                     110
Maryland                      105
Delaware                       96
Minnesota                      89
Connecticut                    82
Missouri                       66
Oklahoma                       66
Alabama                        61
Arkansas                       60

In [15]:
df["State/Province"].nunique()

59

In [16]:
df["Division"].value_counts()

Division
Chocolate    9844
Other         310
Sugar          40
Name: count, dtype: int64

In [17]:
df["Region"].value_counts()

Region
Pacific     3253
Atlantic    2986
Interior    2335
Gulf        1620
Name: count, dtype: int64

In [18]:
df["Product Name"].value_counts()

Product Name
Wonka Bar - Milk Chocolate           2137
Wonka Bar -Scrumdiddlyumptious       2064
Wonka Bar - Triple Dazzle Caramel    2015
Wonka Bar - Fudge Mallows            1818
Wonka Bar - Nutty Crunch Surprise    1810
Wonka Gum                             120
Kazookles                              96
Lickable Wallpaper                     94
SweeTARTS                              10
Laffy Taffy                            10
Fizzy Lifting Drinks                    6
Nerds                                   4
Hair Toffee                             4
Everlasting Gobstopper                  3
Fun Dip                                 3
Name: count, dtype: int64

In [19]:
df["Product Name"].nunique()

15

In [20]:
df["Sales"].describe()

count    10194.000000
mean        13.908537
std         11.341020
min          1.250000
25%          7.200000
50%         10.800000
75%         18.000000
max        260.000000
Name: Sales, dtype: float64

In [21]:
df["Factory"].nunique()

5

In [22]:
df["Customer ID"].nunique()

5044